# String to Variable

In [3]:
# --- matplotlib --------------------------------------------------------------
import matplotlib.pyplot as plt

# --- Our own modules ---------------------------------------------------------
import sys

# permatent location of functions from pyFitXPS
sys.path.insert(0, '/home/julio/Python/pyFitXPS/functions/') # Julio Laptop
#sys.path.insert(0, '/Users/ceciliapatrault/Desktop/Julio/XPS_PYTHON/paquetes_funciones') # Mac
import specs_xy as sp

In [4]:
folder_path = "data/separated/"

In [5]:
files = sorted(os.listdir(folder_path))

In [6]:
for filename in files:
    print(folder_path + filename)

data/separated/Au_MeMBA_5min_10_Survey (0).xy
data/separated/Au_MeMBA_5min_11_S (0).xy
data/separated/Au_MeMBA_5min_12_C (0).xy
data/separated/Au_MeMBA_5min_13_Au (0).xy
data/separated/Au_MeMBA_5min_14_VB (0).xy
data/separated/Au_MeMBA_5min_15_O (0).xy
data/separated/Au_MeMBA_5min_16_S (1).xy
data/separated/Au_MeMBA_5min_17_C (1).xy
data/separated/Au_MeMBA_5min_18_Au (1).xy
data/separated/Au_MeMBA_5min_19_VB (1).xy
data/separated/Au_MeMBA_5min_20_O (1).xy
data/separated/Au_MeMBA_5min_21_S (2).xy
data/separated/Au_MeMBA_5min_22_C (2).xy
data/separated/Au_MeMBA_5min_23_Au (2).xy
data/separated/Au_MeMBA_5min_24_VB (2).xy
data/separated/Au_MeMBA_5min_25_O (2).xy
data/separated/Au_MeMBA_5min_26_S (3).xy
data/separated/Au_MeMBA_5min_27_C (3).xy
data/separated/Au_MeMBA_5min_28_Au (3).xy
data/separated/Au_MeMBA_5min_29_VB (3).xy
data/separated/Au_MeMBA_5min_30_O (3).xy


In [7]:
regiones = sp.load_all_file_xy_in(folder_path)

In [8]:
regiones

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [9]:
from numpy import genfromtxt

In [10]:
filename = 'Au_MeMBA_5min_29_VB (3).xy'

In [11]:
file = open('data/separated/Au_MeMBA_5min_29_VB (3).xy', 'r')
dict_details = {} # create the emty dict to store the details of experiment
        
details_par = ( # parameters to read from comment lines in .xy file
    'Region',
    'Acquisition Date',
    'Analysis Method',
    'Analyzer',
    'Analyzer Lens',
    'Analyzer Slit',
    'Scan Mode',
    'Curves/Scan',
    'Values/Curve',
    'Dwell Time',
    'Excitation Energy',
    'Binding Energy',
    'Pass Energy',
    'Bias Voltage',
    'Detector Voltage',
    'Eff. Workfunction',
    'Source',
    'Comment',
    'Number of Scans',
    )
        
for x in file:
    if '# ' in x:
        z = x.replace("# ", "")
        (k,v) = z.split(":", maxsplit=1) # splir the string after the first ":"
        if k in details_par: # just take the listed keys
            dict_details.update({k.strip(): v.strip()}) # add the k and v to dict
dict_xy = {'details': dict_details, 
           } # create the final dict
        
# key for parameter values to be converted to each proper type
par_float = ('Dwell Time','Excitation Energy','Binding Energy','Pass Energy',
'Bias Voltage','Detector Voltage','Eff. Workfunction','Number of Scans',)
par_int = ('Curves/Scan','Values/Curve','Number of Scans')
        
for keys in par_float:
    dict_xy['details'][keys] = float(dict_xy['details'][keys])
for keys in par_int:
    dict_xy['details'][keys] = int(dict_xy['details'][keys])
        
# closing the opened file
file.close()
        
# read file with numpy.genfromtxt and create one numpy.array pear each column
# and store them as "unwritable" np.array.
BE, intensity = genfromtxt(folder_path + filename, comments='#', unpack=True)
BE = BE * (-1)
KE = dict_xy['details']['Excitation Energy'] + BE
        
# convert BE, KE, intensity for original data as "unwritable" to perserve it
BE.setflags(write=False)
KE.setflags(write=False)
intensity.setflags(write=False)
        
data = {}
data.update({'BE':BE})
data.update({'KE':KE})
data.update({'intensity':intensity})
        
dict_xy.update({'data_orig': data})
conv_str_var = sys.modules[__name__]
setattr(conv_str_var, # _objet
    dict_xy['details']['Region'], # _name: str ... det_dic['region'] = 'region_name'
    dict_xy, # __value: new assigned value to the variable "name"
    )

In [12]:
conv_str_var = sys.modules[__name__]
new_region_name = dict_xy['details']['Region'].replace(" (", "_").replace(")","")
j = setattr(conv_str_var, # _objet
            new_region_name, # _name: str ... det_dic['region'] = 'region_name'
            dict_xy, # __value: new assigned value to the variable "name"
    )

In [13]:
VB_3

{'details': {'Region': 'VB (3)',
  'Acquisition Date': '06/10/22 20:46:44 UTC',
  'Analysis Method': 'XPS',
  'Analyzer Lens': 'MediumArea:3.5kV',
  'Analyzer Slit': '5:7x20\\B:open',
  'Scan Mode': 'FixedAnalyzerTransmission',
  'Curves/Scan': 1,
  'Values/Curve': 281,
  'Dwell Time': 0.1,
  'Excitation Energy': 1486.71,
  'Binding Energy': 12.0,
  'Pass Energy': 20.0,
  'Bias Voltage': 90.0,
  'Detector Voltage': 1750.0,
  'Eff. Workfunction': 4.341,
  'Source': 'Focus',
  'Comment': "15'irrad",
  'Number of Scans': 5},
 'data_orig': {'BE': array([-12.  , -11.95, -11.9 , -11.85, -11.8 , -11.75, -11.7 , -11.65,
         -11.6 , -11.55, -11.5 , -11.45, -11.4 , -11.35, -11.3 , -11.25,
         -11.2 , -11.15, -11.1 , -11.05, -11.  , -10.95, -10.9 , -10.85,
         -10.8 , -10.75, -10.7 , -10.65, -10.6 , -10.55, -10.5 , -10.45,
         -10.4 , -10.35, -10.3 , -10.25, -10.2 , -10.15, -10.1 , -10.05,
         -10.  ,  -9.95,  -9.9 ,  -9.85,  -9.8 ,  -9.75,  -9.7 ,  -9.65,
          -9.6 

In [16]:
print(j)

None
